In [3]:
#Import Library
import numpy as np
import pandas as pd

In [4]:
#Load Dataset
Home_data = pd.read_csv('HomeC.csv')
nan_removed = pd.DataFrame.dropna(pd.DataFrame(Home_data))
nan_removed.value_counts()

C:\Users\M-SI\AppData\Local\Temp\ipykernel_17844\675098868.py:2: DtypeWarning: Columns (0,27) have mixed types. Specify dtype option on import or set low_memory=False.
  Home_data = pd.read_csv('HomeC.csv')


time        use [kW]  gen [kW]  House overall [kW]  Dishwasher [kW]  Furnace 1 [kW]  Furnace 2 [kW]  Home office [kW]  Fridge [kW]  Wine cellar [kW]  Garage door [kW]  Kitchen 12 [kW]  Kitchen 14 [kW]  Kitchen 38 [kW]  Barn [kW]  Well [kW]  Microwave [kW]  Living room [kW]  Solar [kW]  temperature  icon         humidity  visibility  summary     apparentTemperature  pressure  windSpeed  cloudCover  windBearing  precipIntensity  dewPoint  precipProbability
1451624400  0.932833  0.003483  0.932833            0.000033         0.020700        0.061917        0.442633          0.124150     0.006983          0.013083          0.000417         0.000150         0.000000         0.031350   0.001017   0.004067        0.001517          0.003483    36.14        clear-night  0.62      10.00       Clear       29.26                1016.91   9.18       cloudCover  282.0        0.0000           24.40     0.00                 1
1451960350  0.498450  0.054750  0.498450            0.000000         0.020100

In [5]:
# remove duplicates values for temperature, icon, humidity, visibility, summary, apparentTemperature, pressure, windSpeed, cloudCover, windBearing, precipIntensity, dewPoint, and precipProbability
duplicate_removed = nan_removed.drop_duplicates(subset=['temperature', 'icon', 'humidity', 'visibility', 'summary', 'apparentTemperature', 'pressure', 'windSpeed', 'cloudCover', 'windBearing', 'precipIntensity', 'dewPoint', 'precipProbability'], keep='first')
duplicate_removed['summary'].value_counts()

summary
Clear                       6549
Partly Cloudy               1083
Light Rain                   476
Drizzle                      180
Overcast                     105
Rain                          90
Mostly Cloudy                 79
Light Snow                    76
Flurries                      31
Breezy                        27
Snow                          20
Breezy and Partly Cloudy      18
Foggy                         17
Rain and Breezy                3
Heavy Snow                     3
Flurries and Breezy            2
Breezy and Mostly Cloudy       1
Dry                            1
Name: count, dtype: int64

In [6]:
random_selection = duplicate_removed.sample(n=2500, replace=False, random_state=42)
random_selection['summary'].value_counts()

summary
Clear                       1888
Partly Cloudy                309
Light Rain                   120
Drizzle                       45
Overcast                      34
Rain                          30
Mostly Cloudy                 27
Light Snow                    21
Flurries                       9
Snow                           6
Breezy                         3
Breezy and Partly Cloudy       3
Foggy                          2
Rain and Breezy                2
Breezy and Mostly Cloudy       1
Name: count, dtype: int64

In [7]:
# split the dataframe into 3 type of summary Rain, Cloudy, and Clear
rain = duplicate_removed[duplicate_removed['summary'] == 'Rain']
drizzle = duplicate_removed[duplicate_removed['summary'] == 'Drizzle']
overcast = duplicate_removed[duplicate_removed['summary'] == 'Overcast']
# other = random_selection[(random_selection['summary'] != 'Clear') & (random_selection['summary'] != 'Partly Cloudy')]
# other['summary'] = 'Other'

In [8]:
# combine the 3 dataframe into 1
frames = [rain, drizzle, overcast]
result = pd.concat(frames)

# change the summary into numerical value
result['summary'] = result['summary'].map({'Rain': 0, 'Drizzle': 1, 'Overcast': 2})
result = result.reset_index(drop=True)

In [9]:
X = result.iloc[:,[0,5,7,9]].values
y = result['summary'].values

In [10]:
# create hebb rule function
def hebb_rule(X, y, lr, epoch):
    # initialize weights
    w = np.zeros((1, X.shape[1]))
    # initialize bias
    b = 0
    for i in range(epoch):
        for j in range(X.shape[0]):
            # calculate net input
            net_input = np.dot(w, X[j].reshape(-1,1)) + b
            # calculate predicted output
            pred = np.where(net_input>=0.0, 1, 0)
            # update weights and bias
            w += lr * (y[j] - pred) * X[j].reshape(1,-1)
            b += lr * (y[j] - pred)
    return w, b

In [16]:
X = X.astype(float)
y = y.astype(float)
test, bias = hebb_rule(X, y, lr=0.01, epoch=10)
print("test: ", test)
print("bias: ", bias)

test:  [[ 3.49920261e+09  5.24394666e-01  2.41956166e-01 -8.35266660e-02]]
bias:  [[2.41]]
